Dynamic Capabilities

In [0]:
dbutils.widgets.text("filename", "orders")

In [0]:
p_filename=dbutils.widgets.get("filename")

In [0]:
p_filename


### Data Reading


In [0]:
df_source = spark.read.format("parquet").load(f"abfss://source@rtgpadlsgen2.dfs.core.windows.net/{p_filename}")

df_source.count()


Data Reading

In [0]:
df = spark.readStream.format("cloudFiles").option("cloudFiles.format", "parquet")\
        .option("cloudFiles.schemaLocation",f"abfss://bronze@rtgpadlsgen2.dfs.core.windows.net/checkpoint_{p_filename}")\
        .load(f"abfss://source@rtgpadlsgen2.dfs.core.windows.net/{p_filename}")

### Data Writing

In [0]:
df.writeStream.format("parquet")\
    .option("checkpointLocation",f"abfss://bronze@rtgpadlsgen2.dfs.core.windows.net/checkpoint_{p_filename}")\
    .outputMode("append")\
    .option("path",f"abfss://bronze@rtgpadlsgen2.dfs.core.windows.net/{p_filename}")\
    .trigger(once=True)\
    .start()

Checking Tagret


In [0]:
df_bronze = spark.read.format("parquet").load(f"abfss://bronze@rtgpadlsgen2.dfs.core.windows.net/{p_filename}")
df_bronze.display()

In [0]:
common_columns = list(set(df_source.columns).intersection(set(df_bronze.columns)))

df_source_common = df_source.select(common_columns)
df_bronze_common = df_bronze.select(common_columns)

count_source = df_source_common.count()
count_bronze = df_bronze_common.count()

if count_source == count_bronze and df_source_common.subtract(df_bronze_common).count() == 0 and df_bronze_common.subtract(df_source_common).count() == 0:
    result = {"count": count_source, "status": "matched"}
else:
    result = {"count": max(count_source, count_bronze), "status": "not matched"}

display(spark.createDataFrame([result]))